# **SemEval 2025 Task 9: The Food Hazard Detection Challenge**

**Read Data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
import random
import tensorflow as tf
import torch

# Set seed in all libraries
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
torch.manual_seed(42)

In [ ]:
import pandas as pd
data = pd.read_csv('incidents_train.csv', index_col=0, encoding='utf-8')
data.sample(15)

In [ ]:
import pandas as pd
dataprueba= pd.read_csv('incidents_labelled.csv', index_col=0, encoding='utf-8')
dataprueba.sample(15)

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
data.info()

**CATEGORIES OF VARIABLES**


***ST1 Text classification for food hazard prediction, predicting the type of hazard and product.***

In [ ]:
# Identify categorical variables
categorical_variables = ['hazard-category', 'product-category']

# Print the number of unique categories in each variable
for col in categorical_variables:
    print("Variable:", col)
    print("Número de categorías únicas:", data[col].nunique())
    print("Categorías únicas:", data[col].unique())
    print("\n")

In [ ]:
# Identify categorical variables
categorical_variables = ['hazard', 'product']

# Print the number of unique categories in each variable
for col in categorical_variables:
    print("Variable:", col)
    print("Número de categorías únicas:", data[col].nunique())
    print("Categorías únicas:", data[col].unique())
    print("\n")

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv('incidents_train.csv')

# Select the desired column
columna = df['hazard']

# Save the column as a CSV file
columna.to_csv('columna_extraida.csv', index=False, header=True)


Columna guardada exitosamente.


In [ ]:
# Obtain the count
data_counts = data['hazard'].value_counts()
data_counts.head(15)

In [ ]:
# Obtain the count
data_counts_product = data['product-category'].value_counts()
data_counts_product

# **UNBALANCED DATA, RANDOMFOREST AND TF-IDF ARE USED.**

# **TITLE**

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=2024)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
text_clf_lr = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2,5), max_df=0.5, min_df=5)),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42)),
])

In [ ]:
from sklearn.metrics import classification_report, f1_score
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  print(label.upper())
  text_clf_lr.fit(trainset.title, trainset[label])
  lr_predictions = text_clf_lr.predict(testset.title)
  print(f'macro: {f1_score(testset[label], lr_predictions, zero_division=0, average="macro"):.2f}')
  print(f'micro: {f1_score(testset[label], lr_predictions, zero_division=0, average="micro"):.2f}')

In [ ]:
import pandas as pd
dataprueba= pd.read_csv('incidents_labelled.csv', index_col=0, encoding='utf-8')
dataprueba.sample(15)

In [ ]:
from sklearn.metrics import classification_report, f1_score
predictions = {}
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  print(label.upper())
  text_clf_lr.fit(trainset.title, trainset[label])
  predictions[label] = text_clf_lr.predict(dataprueba.title)
  print(f"F1-micro: {f1_score(dataprueba[label], predictions[label], average='micro'):.2f}")
  print(f"F1-macro: {f1_score(dataprueba[label], predictions[label], average='macro'):.2f}")

In [ ]:
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  dataprueba['RF-'+label] = predictions[label]

In [ ]:
from sklearn.metrics import f1_score

def compute_score(hazards_true, products_true, hazards_pred, products_pred):
  # compute f1 for hazards:
  f1_hazards = f1_score(
    hazards_true,
    hazards_pred,
    average='macro'
  )

  # compute f1 for products:
  f1_products = f1_score(
    products_true[hazards_pred == hazards_true],
    products_pred[hazards_pred == hazards_true],
    average='macro'
  )

  return (f1_hazards + f1_products) / 2.

In [ ]:
print('Score Sub-Task 1:', compute_score(dataprueba['hazard-category'], dataprueba['product-category'], dataprueba['RF-hazard-category'], dataprueba['RF-product-category']))
print('Score Sub-Task 2:', compute_score(dataprueba['hazard'], dataprueba['product'], dataprueba['RF-hazard'], dataprueba['RF-product']))

Score Sub-Task 1: 0.8379866951716192
Score Sub-Task 2: 0.7234708580189386


In [ ]:
import pandas as pd
data_valid = pd.read_csv('incidents.csv', index_col=0, encoding='utf-8')
data_valid

#**Predictions**

In [ ]:
# Validation set predictions
predictions = text_clf.predict(data_valid['title'])

# Stores predictions in the dictionary
predictions_valid[f'{label}'] = predictions

print(f"Predicciones para {label} completadas.")

# Predictions
data_valid_predictions = pd.DataFrame(predictions_valid, index=data_valid.index)

# Save predictions
data_valid_predictions.to_csv('submision.csv', index=False)


In [ ]:
df = pd.read_csv('submision.csv')
df

,hazard-category,product-category,hazard,product
0,biological,"meat, egg and dairy products",escherichia coli,ground beef
1,biological,"meat, egg and dairy products",listeria monocytogenes,thermal processed pork meat
2,foreign bodies,"meat, egg and dairy products",plastic fragment,chicken breast
3,allergens,"meat, egg and dairy products",listeria monocytogenes,baklava
4,allergens,ices and desserts,eggs and products thereof,ice cream
...,...,...,...,...
992,allergens,"meat, egg and dairy products",peanuts and products thereof,chicken based products
993,biological,fruits and vegetables,salmonella,dried elder berries
994,biological,seafood,listeria monocytogenes,Fishes not identified
995,foreign bodies,cereals and bakery products,alkaloids,tortilla chips


# **SVM-TITLE**

In [ ]:
data_SMV = pd.read_csv('incidents_train.csv', index_col=0, encoding='utf-8')
data_SMV.sample(15)

In [ ]:
dataprueba_SMV= pd.read_csv('incidents_labelled.csv', index_col=0, encoding='utf-8')
dataprueba_SMV.sample(15)

In [ ]:
trainset, testset = train_test_split(data_SMV, test_size=0.2, random_state=2024)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
text_clf_SMV = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2,5), max_df=0.5, min_df=5)),
    ('clf',  SVC(kernel='linear', degree=3)),
])

In [ ]:
from sklearn.metrics import classification_report, f1_score
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  print(label.upper())
  text_clf_SMV.fit(trainset.title, trainset[label])
  SMV_predictions = text_clf_SMV.predict(testset.title)
  print(f'macro: {f1_score(testset[label], SMV_predictions, zero_division=0, average="macro"):.2f}')
  print(f'micro: {f1_score(testset[label], SMV_predictions, zero_division=0, average="micro"):.2f}')

In [ ]:
from sklearn.metrics import classification_report, f1_score
SMV_predictions= {}
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  print(label.upper())
  text_clf_SMV.fit(trainset.title, trainset[label])
  SMV_predictions[label] = text_clf_SMV.predict(dataprueba_SMV.title)
  print(f"F1-micro: {f1_score(dataprueba_SMV[label], SMV_predictions[label], average='micro'):.2f}")
  print(f"F1-macro: {f1_score(dataprueba_SMV[label], SMV_predictions[label], average='macro'):.2f}")0

In [ ]:
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  dataprueba_SMV['SMV-'+label] = SMV_predictions[label]

In [ ]:
from sklearn.metrics import f1_score

def compute_score(hazards_true, products_true, hazards_pred, products_pred):
  # compute f1 for hazards:
  f1_hazards = f1_score(
    hazards_true,
    hazards_pred,
    average='macro'
  )

  # compute f1 for products:
  f1_products = f1_score(
    products_true[hazards_pred == hazards_true],
    products_pred[hazards_pred == hazards_true],
    average='macro'
  )

  return (f1_hazards + f1_products) / 2.

In [ ]:
print('Score Sub-Task 1:', compute_score(dataprueba_SMV['hazard-category'], dataprueba_SMV['product-category'], dataprueba_SMV['SMV-hazard-category'], dataprueba_SMV['SMV-product-category']))
print('Score Sub-Task 2:', compute_score(dataprueba_SMV['hazard'], dataprueba_SMV['product'], dataprueba_SMV['SMV-hazard'], dataprueba_SMV['SMV-product']))

# **XGBOOST**

In [ ]:
from xgboost import XGBClassifier

# Definir el pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', XGBClassifier(eval_metric='mlogloss'))
])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier

# Initialize LabelEncoder
le = LabelEncoder()

for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  print(label.upper())

  # Fit label encoder on training data and transform labels
  y_train = le.fit_transform(trainset[label])

  # Fit the pipeline with numerical labels
  pipeline.fit(trainset.title, y_train)

  # Handle unseen labels in test data
  try:
    y_test = le.transform(testset[label])
  except ValueError as e:
    print(f"Error: {e}")
    # Option 1: Handle unseen labels (e.g., assign a default value)
    unseen_labels = set(testset[label]) - set(le.classes_)
    for unseen_label in unseen_labels:
      le.classes_ = np.append(le.classes_, unseen_label)
    y_test = le.transform(testset[label])

  XGB_predictions = pipeline.predict(testset.title)
  print(f'macro: {f1_score(y_test, XGB_predictions, zero_division=0, average="macro"):.2f}')
  print(f'micro: {f1_score(y_test, XGB_predictions, zero_division=0, average="micro"):.2f}')

In [ ]:
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder

XGB_predictions= {}
le = LabelEncoder() # Initialize LabelEncoder

for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  print(label.upper())

  # Fit label encoder and transform labels
  y_train = le.fit_transform(trainset[label])

  pipeline.fit(trainset.title, y_train) # Fit with numerical labels

  # Handle unseen labels in test data
  try:
    y_test = le.transform(dataprueba[label])
  except ValueError as e:
    print(f"Error: {e}")
    # Option 1: Handle unseen labels (e.g., assign a default value)
    unseen_labels = set(dataprueba[label]) - set(le.classes_)
    for unseen_label in unseen_labels:
      le.classes_ = np.append(le.classes_, unseen_label)
    y_test = le.transform(dataprueba[label])

  # Assign predictions to XGB_predictions dictionary
  XGB_predictions[label] = pipeline.predict(dataprueba.title)
  print(f"F1-micro: {f1_score(y_test, XGB_predictions[label], average='micro'):.2f}")
  print(f"F1-macro: {f1_score(y_test, XGB_predictions[label], average='macro'):.2f}")

In [ ]:
for label in ('hazard-category', 'product-category', 'hazard', 'product'):
  dataprueba['XGB-'+label] = XGB_predictions[label]

In [ ]:
from sklearn.metrics import f1_score

def compute_score(hazards_true, products_true, hazards_pred, products_pred):
  # compute f1 for hazards:
  f1_hazards = f1_score(
    hazards_true,
    hazards_pred,
    average='macro'
  )

  # compute f1 for products:
  f1_products = f1_score(
    products_true[hazards_pred == hazards_true],
    products_pred[hazards_pred == hazards_true],
    average='macro'
  )

  return (f1_hazards + f1_products) / 2.

In [ ]:
print('Score Sub-Task 1:', compute_score(dataprueba['hazard-category'], dataprueba['product-category'], dataprueba['XGB-hazard-category'], dataprueba['XGB-product-category']))
print('Score Sub-Task 2:', compute_score(dataprueba['hazard'], dataprueba['product'], dataprueba['XGB-hazard'], dataprueba['XGB-product']))